# Example notebook

Here we get the Article classes for all given bio.tools tools.

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from bh24_literature_mining.utils import load_biotools_pub

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

tools_lower = set()

for index, row in tools.iterrows():
    name = row["name"]
    pubmedid = row["biotoolsID"]
    link = row["file_path"]
    # print(f"Name: {name}, PubMed ID: {pubmedid}, Link: {link}")
    tools_lower.add(name.lower())


print(len(tools_lower))

15005


## Example on how to use europepmc_api

In [25]:
from bh24_literature_mining.europepmc_api import EuropePMCClient


client = EuropePMCClient()
# Call bio.tools query and get a list of Article objects
tool_name = "PeptideProphet"
biotools_articles = client.search_mentions(tool_name)
print(len(biotools_articles))

first_article = biotools_articles[0]

print("The first title is:", first_article.title)
print("Type of the first article:", first_article.pubType)
print("This is pmcid:", first_article.pmcid)


relevant_parahraphs = client.get_relevant_paragraphs(first_article.pmcid, tool_name)

print(len(relevant_parahraphs))

from bh24_literature_mining.europepmc_api import process_pmcid

process_pmcid(first_article.pmcid, tool_name, relevant_parahraphs)


841
The first title is: Rescoring Peptide Spectrum Matches: Boosting Proteomics Performance by Integrating Peptide Property Predictors Into Peptide Identification.
Type of the first article: research-article; review; journal article
This is pmcid: PMC11269915
3


[['PMC11269915',
  'Today, most commonly this (unfiltered) list is handed over to post-processing rescoring (PPR) tools, such as PeptideProphet or Percolator, that train machine learning-based pipelines that classify PSMs into correct and incorrect PSMs including false discovery rate (FDR) estimation.',
  [(109, 123, 'PeptideProphet', 'PeptideProphet')]]]

AttributeError: 'EuropePMCClient' object has no attribute 'process_pmcid'

In [ ]:
import spacy
nlp = spacy.load("en_core_sci_sm", disable=["tagger", "parser", "ner", "lemmatizer", "attribute_ruler"])
nlp.add_pipe("sentencizer")

# this is time consuming. The sentenciser takes 1 minute per article on an average.
final_data = []

for pmcid in tqdm(unique_pmcids, desc="Processing PMCIDs"):
    partial_sentences = annotations_df[annotations_df['pmc_id'] == pmcid]['partial_sentence'].tolist()
    segmented_sentences = get_full_text_xml_paragraphs(pmcid, partial_sentences)
    processed_data = process_pmcid(annotations_df, pmcid, segmented_sentences)   
    if processed_data:
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
# Call cites query with a specific PubMed ID and get a list of Article objects
cites_articles = client.search_cites("32109013")
print("The count of the cites articles (within the first 3 pages):", len(cites_articles))


The count of the cites articles (within the first 3 pages): 1000
